# Data pre processing

In [2]:
import pandas as pd 
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np 
import torch.nn.functional as F 

In [98]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [99]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [100]:
def cabin_location(df):
    df['Cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0])
    df['Cabin_number'] = df['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    df['Cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2])
    return df

In [101]:
train_data = cabin_location(train_data)
test_data = cabin_location(test_data)
train_data.drop(['Cabin', 'PassengerId', 'Name'], axis=1, inplace=True)
test_ids = test_data['PassengerId']
test_data.drop(['Cabin', 'PassengerId', 'Name'], axis=1, inplace=True)

In [102]:
from sklearn.calibration import LabelEncoder


label_encoder = LabelEncoder()
for col in train_data.columns:
    if train_data[col].dtype == 'object' or train_data[col].dtype =='bool':
        train_data[col] = label_encoder.fit_transform(train_data[col])
        if col != 'Transported':
            test_data[col] = label_encoder.transform(test_data[col])

In [103]:
test_data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_deck,Cabin_number,Cabin_side
0,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0,6,3,1
1,0,0,2,19.0,0,0.0,9.0,0.0,2823.0,0.0,5,4,1
2,1,1,0,31.0,0,0.0,0.0,0.0,0.0,0.0,2,0,1
3,1,0,2,38.0,0,0.0,6652.0,0.0,181.0,585.0,2,1,1
4,0,0,2,20.0,0,10.0,0.0,635.0,0.0,0.0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269,0,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,6,1495,1
4270,2,0,2,43.0,0,47.0,0.0,3851.0,0.0,0.0,3,278,1
4271,0,0,2,40.0,0,0.0,865.0,0.0,3.0,0.0,5,1796,1
4272,0,1,2,34.0,0,0.0,0.0,0.0,0.0,0.0,6,1496,1


# Splitting into test and train

In [140]:
X_train = train_data.drop(['Transported'], axis=1)
y_train = train_data['Transported']

In [141]:
y_train

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 6606, dtype: int64

In [111]:
y_train = pd.get_dummies(y_train).astype(int)

In [112]:
y_train

,0,1
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1
...,...,...
8688,1,0
8689,1,0
8690,0,1
8691,1,0


In [116]:
from sklearn.model_selection import train_test_split


x_t, x_val, y_t, y_val = train_test_split(X_train, y_train, stratify=y_train,
                                          test_size=0.3, random_state=42)


In [117]:
x_t = torch.tensor(x_t.values, dtype=torch.float32)
y_t = torch.tensor(y_t.values, dtype=torch.float32)
x_val = torch.tensor(x_val.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

In [119]:
dataset = TensorDataset(x_t, y_t)
loader = DataLoader(dataset, batch_size=8)

In [212]:
from typing import Any


class NN:


In [215]:
train_losses = []
val_losses = []
for epoch in range(10):
    running_loss = 0
    for n, batch in enumerate(loader):
        X, y = [x for x in batch]
        
        output = model(X)
        print(output)
        loss = nn.BCELoss()(y, output)
        print(loss.item())
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    train_losses.append(running_loss/(n+1))
    with torch.no_grad():
        output = model(x_val)
        loss = nn.BCELoss()(y_val, output)
        val_losses.append(loss.item())


tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<SigmoidBackward0>)
50.0
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SigmoidBackward0>)


RuntimeError: all elements of target should be between 0 and 1